<div>
<img src="https://discuss.pytorch.org/uploads/default/original/2X/3/35226d9fbc661ced1c5d17e374638389178c3176.png" width="400" style="margin: 50px auto; display: block; position: relative; left: -30px;" />
</div>

<!--NAVIGATION-->
# < [Optimization](3-Optimization.ipynb) | Modules | [CNN](5-CNN.ipynb) >

### PyTorch Modules

Modules are a way to build re-usable model components and to manage model parameters.
PyTorch has many built-in modules for common operations like convolutions, recurrent neural networks, max pooling, common activation functions, etc.
You can also build your own modules.

This notenook introduces modules, and you will build a small neural network to perform image classification on MNIST.

### Table of Contents
#### 1. [Modules](#Modules)
#### 2. [Building a Neural Network](#Building-a-Neural-Network)

---

# Modules

In [ ]:
import torch
import matplotlib.pyplot as plt

### Modules manage parameters

They help to
- keep track of the parameters in your model.
- saving/loading of your model.
- reset gradients (with `model.zero_grad()`)
- moving all parameters to the gpu (with `model.cuda()`)

The model parameter's are represented by `torch.nn.Parameter` objects.
A `Parameter` is a tensor with `requires_grad` to `True` by default, and which is automatically added to the list of parameters when used within a model. Let's have a look at the documentation ([torch.nn.Parameter](https://pytorch.org/docs/stable/_modules/torch/nn/parameter.html))

In [ ]:
print(torch.nn.Parameter.__doc__)

A `Conv1d` module has two parameters: `weight` as `bias`:

In [ ]:
module = torch.nn.Conv1d(5, 2, 3)
print(module.weight)

---

## Basic usage

<div class="alert alert-info">TODO: Split this into cells with some explanation inbetween</div>

In [ ]:
# (1) Create an instance of the model
# each instance has its own parameters
linear_regression_model = torch.nn.Linear(5, 2)  # 5 input dimensions, 2 output dimensions

# Example batch of data_points
#   - models operate on batches of multiple examples together
batch_size = 1
x = torch.randn(batch_size, 5)
y = torch.randn(batch_size, 2)

print("x = {}\ny = {}".format(x, y))

# (2) Run the model
predicted_y = linear_regression_model(x)
print("predicted y = {}".format(predicted_y))

# Compute a loss
loss = torch.sum((y - predicted_y)**2)

# (3) Compute a gradient w.r.t. all model parameters
linear_regression_model.zero_grad()  # clear previous gradient
loss.backward()

print("\nGradients:")
print(linear_regression_model.weight.grad)
print(linear_regression_model.bias.grad)

---

## Composing modules with `torch.nn.Sequential`

If your model is a simple chain of other modules, you can compose them using `torch.nn.Sequential`:

In [ ]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2),
)

# Run the model:
neural_net(x)

---

## Module parameters

You can inspect your network's parameters using `.parameters()` or `.named_parameters()`:

In [ ]:
for param, tensor in neural_net.named_parameters():
    print("{:10s} shape={}".format(param, tensor.shape))

---

## Custom modules

A module has to implement two functions:

- the `__init__` function, where you define all the layers that have learnable parameters. In the `__init__` function, you are just specifying each layer and not how it is connected to others, so it does not need to be in order of execution. Since your model's submodules and parameters are instantiated in the `__init__` function, PyTorch knows that they exist and registers them.  
Also, don't forget to always call the `super()` method.  


- the `forward` function, which is the method that defines what has to be executed during the forward pass and especially how the layers are connected. This is where you call the layers that you defined inside the `__init__` function.


In [ ]:
# This is the most basic form of a custom module:
class MySuperSimpleModule(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        
        # Define sub-modules or parameters
        # torch.nn.Module takes care of adding their parameters to the new module
        self.linear = torch.nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

You can use the print function to list a model's submodules and parameters defined inside `init`:

In [ ]:
model = MySuperSimpleModule(input_size=20, num_classes=5)
print(model)

You can use `model.parameters()` to get the list of parameters of your model automatically inferred by PyTorch.

In [ ]:
for name, p in model.named_parameters():  # Here we use a sligtly different version of the parameters() function
    print(name, ":\n", p)                 # which also returns the parameter name

---

# Building a Neural Network

<div class="alert alert-info">TODO: First introduce the problem / dataset. Also remove the train function and inline it.</div>

__Exercise__<br>
Implement a simple multilayer perceptron with two hidden layers and the following structure:

![](https://raw.githubusercontent.com/ledell/sldm4-h2o/master/mlp_network.png)

We want the following number of neurons in each layer:

- Input-size: *input_size*
- 1st hidden layer: 75
- 2nd hidden layer: 50
- Output layer: *num_classes*

We use `ReLU`s as ‘activation functions’ inbetween each pair of layers.

You will need some PyTorch NN modules - Find them in the [PyTorch doc](https://pytorch.org/docs/master/nn.html) (especially nn.Linear)!

In [ ]:
import torch.nn.functional as F  # provides some helper functions like Relu's, Sigmoids, Tanh, etc.

class MyMultilayerPerceptron(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyMultilayerPerceptron, self).__init__()
        
        self.input_size = input_size
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(input_size, 75)
        self.linear_2 = # <YOUR CODE>
        self.linear_3 = torch.nn.Linear(50, num_classes)
        
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = # <YOUR CODE>
        out = # <YOUR CODE>
        return out

Now feed an input to your network:

In [ ]:
x = torch.rand(16, 784)  # the first dimension is reserved for the 'batch_size'
model = MyMultilayerPerceptron(784, 10)
out = model(x)  # equivalent to model.forward(x)
out[0, :]

---

## Training the model

Most of the functions to train a model follow a similar pattern in PyTorch.
In most of the cases in consists of the following steps:
- Loop over data (in batches)
- Create a prediction (forward pass)
- Clear previous gradients (!)
- Compute gradients (backward pass)
- Parameter update (using an optimizer)

In [ ]:
def train(model, num_epochs, data_loader, device):
    model = model.to(device)
    
    # Define the Loss function and Optimizer that you want to use
    criterion = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # NOTE: model.parameters()
    
    # Outter training loop
    for epoch in range(num_epochs):
        # Inner training loop
        cum_loss = 0
        for (inputs, labels) in data_loader:
            # Prepare inputs and labels for processing by the model (e.g. reshape, move to device, ...)
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # original shape is [batch_size, 28, 28] because it's an image of size 28x28
            inputs = inputs.view(-1, 28*28)

            # Do Forward -> Loss Computation -> Backward -> Optimization
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            cum_loss += loss.item()
        print("Epoch %d, Loss=%.4f" % (epoch+1, cum_loss/len(train_loader)))

Note:
- We can use the `.to` function on the model directly. Indeed, since PyTorch knows all the model parameters, it can put all the parameters on the correct device.
- We use `model.parameters()` to get all the parameters of the model and we can instantiate an optimizer that will optimize these parameters `torch.optim.SGD(model.parameters())`.
- To apply the forward function of the module, we write `model(input)`. In most cases, `model.forward(inputs)` would also work, but there is a slight difference : PyTorch allows you to register hook functions for a model that are automatically called when you do a forward pass on your model. Using `model(input)` will call these hooks and then call the forward function, while using `model.forward(inputs)` will just silently ignore them.

Do you see the convenience of Modules?

### Loss functions

PyTorch comes with a lot of predefined loss functions :
- L1Loss
- MSELoss
- CrossEntropyLoss
- NLLLoss
- PoissonNLLLoss
- KLDivLoss
- BCELoss
- ...

Check out the [PyTorch Documentation](https://pytorch.org/docs/master/nn.html#loss-functions).

---

## The MNIST digit-classification dataset


![MNIST](figures/mnist.jpeg)

First, we have to load the training and test images. MNIST is a widely used dataset, therefore the torchvision package provides simple functionalities to load images from it.

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

batch_size = 64

# MNIST Dataset (Images and Labels)
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# Dataset Loader (Input Batcher)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In PyTorch, `Dataset` and `Dataloaders` are classes that can help to quickly define how to access and iterate over your data. This is specially interesting when your data is distributed over several files (for instance, if you have several images in some directory structure).

### Training the model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyMultilayerPerceptron(input_size=784, num_classes=10)
num_epochs = 5

train(model, num_epochs, train_loader, device)

### Assessing model performance 
This function loops over another `data_loader` (usually containing test/validation data) and computes the model's accuracy on it.

In [ ]:
def accuracy(model, data_loader, device):
    with torch.no_grad(): # during model evaluation, we don't need the autograd mechanism (speeds things up)
        correct = 0
        total = 0
        for inputs, labels in data_loader:
            inputs = inputs.to(device)     
            inputs = inputs.view(-1, 28*28)
            
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            correct += (predicted.cpu() == labels).sum().item()
            total += labels.size(0)
            
    acc = correct / total
    return acc

In [ ]:
accuracy(model, test_loader, device)  # look at: accuracy(model, train_loader, device)

### We get an accuracy of ~97.9%. Can you improve this?

---

## Storing and loading models

In [ ]:
torch.save(model, "my_model.pt")

In [ ]:
my_model_loaded = torch.load("my_model.pt")

In [ ]:
model.linear_3.bias, my_model_loaded.linear_3.bias

___

This intro to modules used [this medium post](https://medium.com/deeplearningbrasilia/deep-learning-introduction-to-pytorch-5bd39421c84) as a resource.

<!--NAVIGATION-->
# < [Optimization](3-Optimization.ipynb) | Modules | [CNN](5-CNN.ipynb) >